## GPT for code dictation

In [20]:
from transformers import AutoTokenizer, TextDataset, DataCollatorForLanguageModeling, AutoModelForCausalLM, pipeline, \
                         Trainer, TrainingArguments
import pandas as pd
from datasets import Dataset


In [21]:
MODEL = 'gpt2'

tokenizer = AutoTokenizer.from_pretrained(MODEL)  # load up a standard gpt2 model

tokenizer.pad_token = tokenizer.eos_token  # set the pad token to avoid a warning


In [22]:
data = pd.read_csv('english_to_latex.csv')

print(data.shape)

data.head(2)

(50, 2)


,English,LaTeX
0,integral from a to b of x squared,"\int_{a}^{b} x^2 \,dx"
1,integral from negative 1 to 1 of x squared,"\int_{-1}^{1} x^2 \,dx"


In [23]:
data.head(10)

,English,LaTeX
0,integral from a to b of x squared,"\int_{a}^{b} x^2 \,dx"
1,integral from negative 1 to 1 of x squared,"\int_{-1}^{1} x^2 \,dx"
2,integral from negative 1 to infinity of x cubed,"\int_{-1}^{\inf} x^3 \,dx"
3,integral from 0 to infinity of x squared,"\int_{0}^{\inf} x^2 \,dx"
4,integral from 0 to infinity of y squared,"\int_{0}^{\inf} y^2 \,dy"
5,integral from 1 to 2 of x over 2,"\int_{1}^{2} \frac{x}{2} \,dx"
6,f of x equals x squared,f(x) = x^2
7,h of x equals x squared,h(x) = x^2
8,g of x equals x squared,g(x) = x^2
9,g of x equals x to the eighth power,g(x) = x^8


In [24]:
# Add our singular prompt
CONVERSION_PROMPT = 'Convert English to LaTeX\n'  # LaTeX conversion task

CONVERSION_TOKEN = 'LaTeX:'


# This is our "training prompt" that we want GPT2 to recognize and learn
training_examples = f'{CONVERSION_PROMPT}English: ' + data['English'] + '\n' + CONVERSION_TOKEN + ' ' + data['LaTeX'].astype(str)

print(training_examples[0])


Convert English to LaTeX
English: integral from a to b of x squared
LaTeX: \int_{a}^{b} x^2 \,dx


In [25]:
task_df = pd.DataFrame({'text': training_examples})

task_df.head(2)

,text
0,Convert English to LaTeX\nEnglish: integral fr...
1,Convert English to LaTeX\nEnglish: integral fr...


In [26]:
# adding the EOS token at the end so the model knows when to stop predicting

task_df['text'] = task_df['text'].map(lambda x: f'{x}{tokenizer.eos_token}')

In [27]:
latex_data = Dataset.from_pandas(task_df)  # turn a pandas DataFrame into a Dataset

def preprocess(examples):  
    # tokenize our text but don't pad because our collator will pad for us dynamically
    return tokenizer(examples['text'], truncation=True)

latex_data = latex_data.map(preprocess, batched=True)

latex_data = latex_data.train_test_split(train_size=.8)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [28]:
latex_data['train'][0]

{'text': 'Convert English to LaTeX\nEnglish: x squared over n\nLaTeX: \\frac{x^2}{n}<|endoftext|>',
 'input_ids': [3103,
  1851,
  3594,
  284,
  4689,
  49568,
  198,
  15823,
  25,
  2124,
  44345,
  625,
  299,
  198,
  14772,
  49568,
  25,
  3467,
  31944,
  90,
  87,
  61,
  17,
  18477,
  77,
  92,
  50256],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [29]:
# standard data collator for auto-regressive language modelling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [30]:
latex_gpt2 = AutoModelForCausalLM.from_pretrained(MODEL)

In [31]:
latex_data

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 40
    })
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 10
    })
})

# Attempt 1 at fine-tuning GPT2 at a LaTeX conversion task

In [34]:
training_args = TrainingArguments(
    output_dir="./english_to_latex",
    overwrite_output_dir=True, # overwrite the content of the output directory
    num_train_epochs=5, # number of training epochs
    per_device_train_batch_size=1, # batch size for training
    per_device_eval_batch_size=20,  # batch size for evaluation
    load_best_model_at_end=True,
    logging_steps=5,
    log_level='info',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    #use_mps_device=True
)

trainer = Trainer(
    model=latex_gpt2,
    args=training_args,
    train_dataset=latex_data["train"],
    eval_dataset=latex_data["test"],
    data_collator=data_collator,
)

trainer.evaluate()

C:\Users\Archit\anaconda3\envs\latexgpt\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 20


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


{'eval_loss': 0.6889172792434692,
 'eval_runtime': 0.6817,
 'eval_samples_per_second': 14.669,
 'eval_steps_per_second': 1.467}

In [35]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 40
  Num Epochs = 5
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 200
  Number of trainable parameters = 124,439,808


Epoch,Training Loss,Validation Loss
1,0.606500,0.740068
2,0.346400,0.772659
3,0.490100,0.702616
4,0.388000,0.654477
5,0.318700,0.621575


The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 20
Saving model checkpoint to ./english_to_latex\checkpoint-40
Configuration saved in ./english_to_latex\checkpoint-40\config.json
Configuration saved in ./english_to_latex\checkpoint-40\generation_config.json
Model weights saved in ./english_to_latex\checkpoint-40\model.safetensors
C:\Users\Archit\anaconda3\envs\latexgpt\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadMod

TrainOutput(global_step=200, training_loss=0.40061594724655153, metrics={'train_runtime': 215.4959, 'train_samples_per_second': 0.928, 'train_steps_per_second': 0.928, 'total_flos': 3092636160000.0, 'train_loss': 0.40061594724655153, 'epoch': 5.0})

In [37]:
book_data = TextDataset(
    tokenizer=tokenizer,
    file_path='latex-guide-cos423.txt',  # train on a LaTeX cheat sheet they made
    block_size=128
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,  # MLM is Masked Language Modelling
)

latex_gpt2 = AutoModelForCausalLM.from_pretrained(MODEL)

training_args = TrainingArguments(
    output_dir="./math_book",
    overwrite_output_dir=True, # overwrite the content of the output directory
    num_train_epochs=10, # number of training epochs
    per_device_train_batch_size=2, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    load_best_model_at_end=True,
    logging_steps=10,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    #use_mps_device=True
)

trainer = Trainer(
    model=latex_gpt2,
    args=training_args,
    data_collator=data_collator,
    train_dataset=book_data.examples[:int(len(book_data.examples)*.8)],
    eval_dataset=book_data.examples[int(len(book_data.examples)*.8):]
)

Loading features from cached file cached_lm_GPT2TokenizerFast_128_latex-guide-cos423.txt [took 0.006 s]
loading configuration file config.json from cache at C:\Users\Archit\.cache\huggingface\hub\models--gpt2\snapshots\607a30d783dfa663caf39e06633721c8d4cfcd7e\config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_p

In [38]:
trainer.evaluate()  # initial loss for the cheat sheet

***** Running Evaluation *****
  Num examples = 12
  Batch size = 32
C:\Users\Archit\anaconda3\envs\latexgpt\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


{'eval_loss': 2.3705146312713623,
 'eval_runtime': 1.8117,
 'eval_samples_per_second': 6.624,
 'eval_steps_per_second': 0.552}

In [39]:
trainer.train()

***** Running training *****
  Num examples = 47
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 240
  Number of trainable parameters = 124,439,808


Epoch,Training Loss,Validation Loss
1,1.711400,1.825186
2,1.339100,1.779840
3,1.420700,1.773209
4,1.145400,1.804101
5,1.210000,1.856383
6,0.837000,1.906700
7,0.893400,1.938794
8,0.747700,1.983484
9,0.725900,1.985252
10,0.777600,1.995426


***** Running Evaluation *****
  Num examples = 12
  Batch size = 32
Saving model checkpoint to ./math_book\checkpoint-24
Configuration saved in ./math_book\checkpoint-24\config.json
Configuration saved in ./math_book\checkpoint-24\generation_config.json
Model weights saved in ./math_book\checkpoint-24\model.safetensors
C:\Users\Archit\anaconda3\envs\latexgpt\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
***** Running Evaluation *****
  Num examples = 12
  Batch size = 32
Saving model checkpoint to ./math_book\checkpoint-48
Configuration saved in ./math_book\checkpoint-48\config.json
Configuration saved in ./math_book\checkpoint-48\generation_config.json
Model weights saved in ./math_book\checkpoint-48\model.safetensors
C:\Users\Archit\anaconda3\envs\latexgpt\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memo

TrainOutput(global_step=240, training_loss=1.1517807781696319, metrics={'train_runtime': 578.4861, 'train_samples_per_second': 0.812, 'train_steps_per_second': 0.415, 'total_flos': 30701813760000.0, 'train_loss': 1.1517807781696319, 'epoch': 10.0})

In [40]:
trainer.save_model()

Saving model checkpoint to ./math_book
Configuration saved in ./math_book\config.json
Configuration saved in ./math_book\generation_config.json
Model weights saved in ./math_book\model.safetensors


# Restart training now with our own pre-trained "foundation" model

In [41]:
# load up our gpt pre-trained on latex cheat sheets
math_latex_gpt2 = AutoModelForCausalLM.from_pretrained('./math_book')

training_args = TrainingArguments(
    output_dir="./math_english_to_latex",
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=5, # number of training epochs
    per_device_train_batch_size=1, # batch size for training
    per_device_eval_batch_size=20,  # batch size for evaluation
    load_best_model_at_end=True,
    logging_steps=5,
    log_level='info',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    #use_mps_device=True
)

trainer = Trainer(
    model=math_latex_gpt2,
    args=training_args,
    train_dataset=latex_data["train"],
    eval_dataset=latex_data["test"],
    data_collator=data_collator,
)

trainer.evaluate()  # loss is starting slightly lower than before

loading configuration file ./math_book\config.json
Model config GPT2Config {
  "_name_or_path": "./math_book",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.41.1",
  "use_cache": true,
  "vocab_size": 50257
}

loading we

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


{'eval_loss': 4.079037666320801,
 'eval_runtime': 0.704,
 'eval_samples_per_second': 14.205,
 'eval_steps_per_second': 1.42}

In [42]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 40
  Num Epochs = 5
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 200
  Number of trainable parameters = 124,439,808


Epoch,Training Loss,Validation Loss
1,1.103400,0.952675
2,0.637500,0.758987
3,0.610600,0.709967
4,0.425500,0.649072
5,0.520000,0.629295


The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 20
Saving model checkpoint to ./math_english_to_latex\checkpoint-40
Configuration saved in ./math_english_to_latex\checkpoint-40\config.json
Configuration saved in ./math_english_to_latex\checkpoint-40\generation_config.json
Model weights saved in ./math_english_to_latex\checkpoint-40\model.safetensors
C:\Users\Archit\anaconda3\envs\latexgpt\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expect

TrainOutput(global_step=200, training_loss=0.7976289844512939, metrics={'train_runtime': 201.1583, 'train_samples_per_second': 0.994, 'train_steps_per_second': 0.994, 'total_flos': 3092636160000.0, 'train_loss': 0.7976289844512939, 'epoch': 5.0})

In [43]:
trainer.save_model()  # save this model

Saving model checkpoint to ./math_english_to_latex
Configuration saved in ./math_english_to_latex\config.json
Configuration saved in ./math_english_to_latex\generation_config.json
Model weights saved in ./math_english_to_latex\model.safetensors


In [44]:
loaded_model = AutoModelForCausalLM.from_pretrained('./math_english_to_latex')
latex_generator = pipeline('text-generation', model=loaded_model, tokenizer=tokenizer)

text_sample = 'g of x equals integral from 0 to 1 of x squared'
conversion_text_sample = f'{CONVERSION_PROMPT}English: {text_sample}\n{CONVERSION_TOKEN}'

print(latex_generator(
    conversion_text_sample, num_beams=2, early_stopping=True, temperature=0.7,
    max_new_tokens=24
)[0]['generated_text'])

loading configuration file ./math_english_to_latex\config.json
Model config GPT2Config {
  "_name_or_path": "./math_english_to_latex",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.41.1",
  "use_cache": true,
  "vocab_si

Convert English to LaTeX
English: g of x equals integral from 0 to 1 of x squared
LaTeX: g(x) = \int_{0}^{1} x^2 \,dx^2 \,dx


In [45]:
# Another example
text_sample = 'r of x is sum from 0 to x of x squared'
conversion_text_sample = f'{CONVERSION_PROMPT}English: {text_sample}\n{CONVERSION_TOKEN}'

print(latex_generator(
    conversion_text_sample, num_beams=5, early_stopping=True, temperature=0.7,
    max_length=len(tokenizer.encode(conversion_text_sample)) + 20
)[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Convert English to LaTeX
English: r of x is sum from 0 to x of x squared
LaTeX: \sum_{0}^{x} x^2 \,dx^2 \,dx^


In [46]:
print(latex_generator(
    text_sample, num_beams=5, early_stopping=True, temperature=0.7,
    max_length=len(tokenizer.encode(conversion_text_sample)) + 20
)[0]['generated_text'])

r of x is sum from 0 to x of x squared
English: \sum_{0}^{x} x^2 \,dx^2 \,dx^2 \,dx^2 \,dx^


In [78]:
# try a few shot with standard gpt2
few_shot_prompt = CONVERSION_PROMPT+"""English: f of x is sum from 0 to x of x squared
LaTeX: f(x) = \sum_{0}^{x} x^2 \,dx \
###
LCT
English: f of x equals integral from 0 to pi of x to the fourth power
LaTeX: f(x) = \int_{0}^{\pi} x^4 \,dx \
###
LCT
English: pi to the 8th power
LaTeX:"""

print(non_finetuned_latex_generator(
    few_shot_prompt, num_beams=1, early_stopping=True, temperature=0.1,
    max_length=len(tokenizer.encode(few_shot_prompt)) + 20
)[0]['generated_text'])

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "do_sample": true,
  "eos_token_id": 50256,
  "max_length": 50,
  "transformers_version": "4.30.2"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Convert English to LaTeX
English: f of x is sum from 0 to x of x squared
LaTeX: f(x) = \sum_{0}^{x} x^2 \,dx ###
LCT
English: f of x equals integral from 0 to pi of x to the fourth power
LaTeX: f(x) = \int_{0}^{\pi} x^4 \,dx ###
LCT
English: pi to the 8th power
LaTeX: pi to the 8th power
LCT
English: f(x) = \sum_{


In [80]:
# Just ask with standard gpt2
print(non_finetuned_latex_generator(
    conversion_text_sample, num_beams=1, early_stopping=True, temperature=0.1,
    max_length=len(tokenizer.encode(conversion_text_sample)) + 20
)[0]['generated_text'])

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "do_sample": true,
  "eos_token_id": 50256,
  "max_length": 50,
  "transformers_version": "4.30.2"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Convert English to LaTeX
English: pi to the 16th power
LaTeX: pi to the 16th power
LaTeX: pi to the 16th power
LaTeX:
